In [1]:
import pandas as pd

In [24]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
df=pd.read_csv("d:/caravan.csv")

In [5]:
df.head()

,Unnamed: 0,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,...,APERSONG,AGEZONG,AWAOREG,ABRAND,AZEILPL,APLEZIER,AFIETS,AINBOED,ABYSTAND,Purchase
0,1,33,1,3,2,8,0,5,1,3,...,0,0,0,1,0,0,0,0,0,No
1,2,37,1,2,2,8,1,4,1,4,...,0,0,0,1,0,0,0,0,0,No
2,3,37,1,2,2,8,0,4,2,4,...,0,0,0,1,0,0,0,0,0,No
3,4,9,1,3,3,3,2,3,2,4,...,0,0,0,1,0,0,0,0,0,No
4,5,40,1,4,2,10,1,4,1,4,...,0,0,0,1,0,0,0,0,0,No


In [6]:
x=df.drop("Purchase",axis=1)

In [7]:
y=df["Purchase"]

In [10]:
x_tr,x_test,y_tr,y_test=train_test_split(x,y,test_size=.4)

In [11]:
from sklearn import tree

In [17]:
dtree=tree.DecisionTreeClassifier(criterion="entropy")

In [18]:
dtree.fit(x_tr,y_tr)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [41]:
import graphviz
import pydot
with open("caravan.dot", "w") as f:
    f = tree.export_graphviz(dtree, out_file=f)
(graph,) = pydot.graph_from_dot_file('caravan.dot')
graph.write_png("caravan.png")

In [23]:
p_caravan=dtree.predict(x_test)

In [25]:
table=confusion_matrix(y_test,p_caravan)

In [26]:
print(table)

[[2066  118]
 [ 130   15]]


In [28]:
dtree1=tree.DecisionTreeClassifier(criterion="gini")

In [29]:
dtree1.fit(x_tr,y_tr)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [31]:
p_caravan1=dtree1.predict(x_test)

In [33]:
table2=confusion_matrix(y_test,p_caravan1)

In [34]:
print(table2)

[[2063  121]
 [ 132   13]]


In [38]:
#accuracy score
from sklearn.metrics import accuracy_score
accuracy_entropy = accuracy_score(y_test, p_caravan)
accuracy_gini = accuracy_score(y_test, p_caravan1)
print(accuracy_entropy)
print(accuracy_gini)

0.8935165306998711
0.8913696865607557


In [40]:
sum1=0
sum2=0
for i in range(0,len(table)):
    for j in range(0,len(table[0])):
        if i==j:
            sum1=sum1+table[i][j]
        sum2=sum2+table[i][j]
print(sum1)
print(sum2)
Accuracy=sum1/sum2*100
print("accuracy:",Accuracy)


        

2081
2329
accuracy: 89.35165306998711


In [55]:
#printing dtree2
import graphviz
import pydot
with open("caravan2.dot", "w") as f:
    f = tree.export_graphviz(dtree1, out_file=f,feature_names=x_tr.columns)
(graph,) = pydot.graph_from_dot_file('caravan2.dot')
graph.write_png("caravan2.png")

In [46]:
#applying random classifier
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier()
rfc_model.fit(x_tr,y_tr)

C:\Users\INDRANI\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [49]:
p_caravan2=rfc_model.predict(x_test)
table2=confusion_matrix(y_test,p_caravan2)
print(table2)
                        

[[2165   19]
 [ 138    7]]


In [50]:
print("random forest accuracy_core",accuracy_score(y_test,p_caravan2))

random forest accuracy_core 0.9325890940317733


In [68]:
# only cross validation
from sklearn.model_selection import cross_val_score,cross_val_predict
#model dtree1
scores=cross_val_score(dtree1,x,y,cv=5,scoring='f1_micro')
print(scores)


[0.76824034 0.89699571 0.90472103 0.8943299  0.07136715]


In [72]:
#prediction
p=cross_val_predict(dtree1,x,y,cv=5)
table_cross=confusion_matrix(y,p)
print(table_cross)



[[3986 1488]
 [ 238  110]]


In [73]:
print("cross validation accuracy_core",accuracy_score(y,p))

cross validation accuracy_core 0.7035383029886637


In [81]:
#cross validation with bagging
from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
kfold_model=model_selection.KFold(n_splits=5,random_state=10)
dtree3=tree.DecisionTreeClassifier()
n_trees=100
model=BaggingClassifier(base_estimator=dtree3,n_estimators=10,random_state=10)
accuracy=model_selection.cross_val_score(model,x,y,cv=kfold_model)
print(accuracy)
print(accuracy.mean())


[0.92360515 0.93304721 0.93041237 0.93041237 0.92353952]
0.9282033243366813
